In [247]:
from crewai import Agent, Task, LLM , Crew
import os
import agentops

from dotenv import load_dotenv
from fpdf import FPDF
from PyPDF2 import PdfReader
import re




In [248]:
load_dotenv()  # reads .env into environment

groq_key = os.getenv("GROQ_API_KEY")
agentops_key = os.getenv("AGENTOPS_API_KEY")

In [249]:

# agentops.init(api_key=agentops_key, auto_start_session=True)


In [250]:
outputDir = "./output"
os.makedirs(outputDir, exist_ok=True)

In [251]:
basic_llm = LLM(model="groq/llama-3.1-8b-instant", temperature=0.7, api_key=groq_key)

## Summarizer Agent

In [252]:
def read_pdf(file_path):
    reader = PdfReader(file_path)
    return "\n".join(page.extract_text() for page in reader.pages if page.extract_text())

pdf_text = read_pdf("input/sample.pdf")


In [253]:
summarizer_agent = Agent(
    role = "Summarizer Agent",
    goal = "To provide a concise summary of any given document or text, ensuring clarity, accuracy, and output in a user-friendly PDF format.",
    
    backstory = "\n".join([
            "The agent is designed to assist users in quickly understanding the main points of lengthy documents or articles.", 
            "It should distill complex information into easily digestible summaries while maintaining the original context and meaning.", 
            "The agent can also summarize weekly thoughts, reflections, and tasks to support productivity.",
            "The agent should generate a PDF file containing the summary and save it to the output directory.",
            "The agent should support multiple languages and be able to summarize content in the user's preferred language.",
            "The agent should handle various document formats, including PDFs, articles, and plain text.",
    ]),
    
    llm = basic_llm,
    verbose=True
    )


summarizer_task = Task(
    description =  f"""
        Summarize the following PDF text:{pdf_text}

        Read through the document carefully and identify the key points, main ideas, and essential information.
        Create a concise summary that captures the essence of the original text while omitting unnecessary details.
        Ensure that the summary is clear, coherent, and easy to understand.
        Generate a PDF file containing the summary and save it to the output directory.
        """,
        
    expected_output= "\n".join([
        "A well-structured summary containing a concise and accurate summary of the provided document in PDF format.",
        "The summary should be saved as a PDF file in the output directory.",
        "The summary should be clear and concise summary, formatted in short paragraphs or bullet points for easy reading."
        ]),
    agent=summarizer_agent,
    )

output_file_path = os.path.join(outputDir, "summary.pdf")

### Running the Agent

In [254]:
multi_agents = Crew(
    agents=[summarizer_agent],
    tasks=[summarizer_task],
)

In [255]:
result = multi_agents.kickoff({
    "task": "Summarize this PDF",
    "input": pdf_text,
    "output_dir": "output/",
})

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Summarizer Agent                                                                                        │
│                                                                                                                 │
│  Task:                                                                                                          │
│          Summarize the following PDF text:                                                                      │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  DEBI Robot ics Challenge  2023                                                                                 │
│  Rules and Guidelines                                                                                           │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  Sponsored by:       In Cooperation with:                                                                       │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  2/4                                                                                                            │
│                                                                                                                 │
│  Challenge  Description                                                                                         │
│  In continuation of the efforts of Digital Egypt Builders Imitative (DEBI)1, launched by the Egyptian Ministry  │
│  of                                                                                                             │
│  Communications and Information Technology , the DEBI’s Robotics & IoT Track  launches the annual               │
│  DEBI Robot ics Challenge . Inspired by the GdR TurtleB

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Summarizer Agent                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  DEBI Robotics Challenge 2023                                                                                   │
│                                                                                                                 │
│  DEBI Robotics Challenge 2023 is an annual competition launched by the Egyptian Ministry of Communications and  │
│  Information Technology, under the Digital Egypt Builders Initiative (DEBI). The challenge is inspired by the   │
│  GdR TurtleBot Challenge 2018 and aims to advance robotic software and autonomous capabilities. The             │
│  competition is a physical tournament where robots compete in one-vs-one matches in a predefined playground.    │
│                                                                                                                 │
│  Competition Details                                                                                            │
│                                                                                                                 │
│  * The playground is divided into two identical halves separated by a red line.                                 │
│  * Each half has three randomly placed balls with random colors.                                                │
│  * The main mission for each team is to reduce the number of balls in their area by moving them to the          │
│  opponent's side.                                                                                               │
│  * The competition is based on knockout matches, and the winner is determined by the team with fewer balls at   │
│  the end of each match.                                                                                         │
│  * The robot must be fully autonomous, and it cannot step onto or pass the red line or the playground walls.    │
│                                                                                                                 │
│  Rules and Guidelines                                                                                           │
│                                                                                                                 │
│  * All team members must be physically present on the competition day.                                          │
│  * The competition is based on knockout matches, and DEBI reserves the right to change the competition style    │
│  based on the number of participating teams.                                                                    │
│  * The robot must be fully autonomous, and it cannot touch the red line.                                        │
│  * The match duration is 10 minutes, and if no team wins, the match will resume for another 3 minutes.          │
│  * If the match ends in a draw after the first 13 minutes, a golden-ball scheme will be followed.               │
│                                                                                                                 │
│  Eligibility Criteria and Teams Formation                                                                       │
│                                                                                                                 │
│  * All participants must hold Egyptian nationality.                                                             │
│  * All participants must be registered undergraduate st

In [256]:

# Save result to PDF

summary_text = result.raw   # string output

pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)

# Split summary text into lines
for line in summary_text.split("\n"):
    pdf.multi_cell(0, 10, line)

# Save to output folder
output_file_path = os.path.join("output", "summary.pdf")
os.makedirs("output", exist_ok=True)
pdf.output(output_file_path)


print(f"Summary saved at {os.path.abspath(output_file_path)}")

Summary saved at d:\AI\Projects\Multi-agent_Weekly_Manager\output\summary.pdf
